In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covid19-global-forecasting-week-3/train.csv
/kaggle/input/covid19-global-forecasting-week-3/submission.csv
/kaggle/input/covid19-global-forecasting-week-3/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder

In [3]:
train_df=pd.read_csv("../input/covid19-global-forecasting-week-3/train.csv")
test_df=pd.read_csv("../input/covid19-global-forecasting-week-3/test.csv")

In [4]:
train_df.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23562 entries, 0 to 23561
Data columns (total 6 columns):
Id                23562 non-null int64
Province_State    10010 non-null object
Country_Region    23562 non-null object
Date              23562 non-null object
ConfirmedCases    23562 non-null float64
Fatalities        23562 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ MB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13158 entries, 0 to 13157
Data columns (total 4 columns):
ForecastId        13158 non-null int64
Province_State    5590 non-null object
Country_Region    13158 non-null object
Date              13158 non-null object
dtypes: int64(1), object(3)
memory usage: 411.3+ KB


In [7]:
test_df.head()

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-03-26
1,2,NaN,Afghanistan,2020-03-27
2,3,NaN,Afghanistan,2020-03-28
3,4,NaN,Afghanistan,2020-03-29
4,5,NaN,Afghanistan,2020-03-30


In [8]:
print("Min train date: ",train_df["Date"].min())
print("Max train date: ",train_df["Date"].max())
print("Min test date: ",test_df["Date"].min())
print("Max test date: ",test_df["Date"].max())

Min train date:  2020-01-22
Max train date:  2020-04-07
Min test date:  2020-03-26
Max test date:  2020-05-07


In [9]:
train_df.isnull().sum()

Id                    0
Province_State    13552
Country_Region        0
Date                  0
ConfirmedCases        0
Fatalities            0
dtype: int64

In [10]:
test_df.isnull().sum()

ForecastId           0
Province_State    7568
Country_Region       0
Date                 0
dtype: int64

In [11]:
train_df.rename(columns={'Country_Region':'Country'}, inplace=True)
test_df.rename(columns={'Country_Region':'Country'}, inplace=True)

train_df.rename(columns={'Province_State':'State'}, inplace=True)
test_df.rename(columns={'Province_State':'State'}, inplace=True)

In [12]:
mo = train_df['Date'].apply(lambda x: x[5:7])
da = train_df['Date'].apply(lambda x: x[8:10])
mo_test = test_df['Date'].apply(lambda x: x[5:7])
da_test = test_df['Date'].apply(lambda x: x[8:10])
train_df['day_from_jan_first'] = (da.apply(int)
                               + 31*(mo=='02') 
                               + 60*(mo=='03')
                               + 91*(mo=='04')  
                              )
test_df['day_from_jan_first'] = (da_test.apply(int)
                               + 31*(mo_test=='02') 
                               + 60*(mo_test=='03')
                               + 91*(mo_test=='04')  
                              )

In [13]:
train_df["Date"] = train_df["Date"].apply(lambda x:x.replace("-",""))
train_df["Date"]  = train_df["Date"].astype(int)

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23562 entries, 0 to 23561
Data columns (total 7 columns):
Id                    23562 non-null int64
State                 10010 non-null object
Country               23562 non-null object
Date                  23562 non-null int64
ConfirmedCases        23562 non-null float64
Fatalities            23562 non-null float64
day_from_jan_first    23562 non-null int64
dtypes: float64(2), int64(3), object(2)
memory usage: 1.3+ MB


In [15]:
test_df["Date"] = test_df["Date"].apply(lambda x:x.replace("-",""))
test_df["Date"]  = test_df["Date"].astype(int)

In [16]:
EMPTY_VAL = "EMPTY_VAL"

def fillState(state, country):
    if state == EMPTY_VAL: return country
    return state

In [17]:
train_copy = train_df.copy()

In [18]:
train_copy['State'].fillna(EMPTY_VAL, inplace=True)
train_copy['State'] = train_copy.loc[:, ['State', 'Country']].apply(lambda x : fillState(x['State'], x['Country']), axis=1)

In [19]:
train_copy.head()

,Id,State,Country,Date,ConfirmedCases,Fatalities,day_from_jan_first
0,1,Afghanistan,Afghanistan,20200122,0.0,0.0,22
1,2,Afghanistan,Afghanistan,20200123,0.0,0.0,23
2,3,Afghanistan,Afghanistan,20200124,0.0,0.0,24
3,4,Afghanistan,Afghanistan,20200125,0.0,0.0,25
4,5,Afghanistan,Afghanistan,20200126,0.0,0.0,26


In [20]:
test_copy = test_df.copy()

In [21]:
test_copy['State'].fillna(EMPTY_VAL, inplace=True)
test_copy['State'] = test_copy.loc[:, ['State', 'Country']].apply(lambda x : fillState(x['State'], x['Country']), axis=1)

In [22]:
test_copy.head()

,ForecastId,State,Country,Date,day_from_jan_first
0,1,Afghanistan,Afghanistan,20200326,86
1,2,Afghanistan,Afghanistan,20200327,87
2,3,Afghanistan,Afghanistan,20200328,88
3,4,Afghanistan,Afghanistan,20200329,89
4,5,Afghanistan,Afghanistan,20200330,90


In [23]:
labelencoder = LabelEncoder()

In [24]:
train_copy['Country'] = labelencoder.fit_transform(train_copy['Country'])
train_copy['State'] = labelencoder.fit_transform(train_copy['State'])

In [25]:
test_copy['Country'] = labelencoder.fit_transform(test_copy['Country'])
test_copy['State'] = labelencoder.fit_transform(test_copy['State'])

In [26]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23562 entries, 0 to 23561
Data columns (total 7 columns):
Id                    23562 non-null int64
State                 23562 non-null int64
Country               23562 non-null int64
Date                  23562 non-null int64
ConfirmedCases        23562 non-null float64
Fatalities            23562 non-null float64
day_from_jan_first    23562 non-null int64
dtypes: float64(2), int64(5)
memory usage: 1.3 MB


In [27]:
train_copy.columns

Index(['Id', 'State', 'Country', 'Date', 'ConfirmedCases', 'Fatalities',
       'day_from_jan_first'],
      dtype='object')

In [28]:
X=train_copy[['State', 'Country', 'Date', 'day_from_jan_first']]

In [29]:
y1=train_copy["ConfirmedCases"] #Confirmed Case
y2=train_copy["Fatalities"]     #Fatalities

In [30]:
#Confirmed Cases
X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(X, y1, test_size = .20, random_state = 42)

In [31]:
dt1=DecisionTreeRegressor(criterion="friedman_mse",max_depth=20,random_state=42)

In [32]:
dt1.fit(X_train_confirmed, y_train_confirmed)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='friedman_mse', max_depth=20,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [33]:
y_pred_dt_confirmed=dt1.predict(X_test_confirmed)

In [34]:
np.sqrt(mean_squared_log_error( y_test_confirmed, y_pred_dt_confirmed ))

0.3716509454373332

In [35]:
#Fatalities
X_train_fatal, X_test_fatal, y_train_fatal, y_test_fatal = train_test_split(X, y2, test_size = .20, random_state = 42)

In [36]:
dt2=DecisionTreeRegressor(criterion="friedman_mse",max_depth=20,random_state=42)

In [37]:
dt2.fit(X_train_fatal, y_train_fatal)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='friedman_mse', max_depth=20,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [38]:
y_pred_dt_fatal=dt2.predict(X_test_fatal)

In [39]:
np.sqrt(mean_squared_log_error( y_test_fatal, y_pred_dt_fatal ))

0.20351148162400395

In [40]:
test_copy.head()

,ForecastId,State,Country,Date,day_from_jan_first
0,1,0,0,20200326,86
1,2,0,0,20200327,87
2,3,0,0,20200328,88
3,4,0,0,20200329,89
4,5,0,0,20200330,90


In [41]:
X_test=test_copy[['State','Country','Date','day_from_jan_first']]

In [42]:
y_confirmed=dt1.predict(X_test)

In [43]:
y_fatal=dt2.predict(X_test)

In [44]:
submission=pd.DataFrame({'ForecastId': test_copy["ForecastId"], 'ConfirmedCases': y_confirmed, 'Fatalities': y_fatal})

In [45]:
submission.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,94.0,4.0
1,2,110.0,4.0
2,3,110.0,4.0
3,4,120.0,4.0
4,5,170.0,4.0


In [46]:
submission.to_csv('submission.csv', index=False)